# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [2]:
import os
path_do_uk_data = './cc.uk.300.vec.gz'
if not os.path.exists(path_do_uk_data):
    print("Dataset not found locally. Downloading from github. Loading special files as well")
    !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz -nc
    path_do_uk_data = './cc.uk.300.vec.gz'



Dataset not found locally. Downloading from github. Loading special files as well
--2022-12-11 10:53:42--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1257595219 (1.2G) [binary/octet-stream]
Saving to: ‘cc.uk.300.vec.gz’

cc.uk.300.vec.gz    100%[===================>]   1.17G  17.8MB/s    in 65s     

2022-12-11 10:54:49 (18.3 MB/s) - ‘cc.uk.300.vec.gz’ saved [1257595219/1257595219]



In [3]:
path_do_ru_data = './cc.ru.300.vec.gz'
if not os.path.exists(path_do_ru_data):
    print("Dataset not found locally. Downloading from github. Loading special files as well")
    !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz -nc
    path_do_ru_data = './cc.ru.300.vec.gz'

Dataset not found locally. Downloading from github. Loading special files as well
--2022-12-11 10:55:15--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  17.3MB/s    in 64s     

2022-12-11 10:56:20 (19.5 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]



In [4]:
uk_emb = KeyedVectors.load_word2vec_format(path_do_uk_data)

In [5]:
ru_emb = KeyedVectors.load_word2vec_format(path_do_ru_data)

In [6]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [176]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [9]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [10]:
!wget -O ukr_rus.train.txt https://raw.githubusercontent.com/nikhovas/ml-labs-bad-data-fix/main/ukr_rus.train.txt

--2022-12-11 11:08:29--  https://raw.githubusercontent.com/nikhovas/ml-labs-bad-data-fix/main/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Saving to: ‘ukr_rus.train.txt’

ukr_rus.train.txt   100%[===================>]  57.96K  --.-KB/s    in 0.01s   

2022-12-11 11:08:29 (4.23 MB/s) - ‘ukr_rus.train.txt’ saved [59351/59351]



In [11]:
!wget -O ukr_rus.test.txt https://raw.githubusercontent.com/nikhovas/ml-labs-bad-data-fix/main/ukr_rus.test.txt

--2022-12-11 11:08:32--  https://raw.githubusercontent.com/nikhovas/ml-labs-bad-data-fix/main/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Saving to: ‘ukr_rus.test.txt’

ukr_rus.test.txt    100%[===================>]  11.90K  --.-KB/s    in 0s      

2022-12-11 11:08:33 (23.5 MB/s) - ‘ukr_rus.test.txt’ saved [12188/12188]



In [12]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [13]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [17]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression()
mapping.fit(X_train, Y_train)
# -------

LinearRegression()

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [54]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531402945518494),
 ('июнь', 0.8402307629585266),
 ('март', 0.8385775089263916),
 ('сентябрь', 0.8331868052482605),
 ('февраль', 0.8311493992805481),
 ('октябрь', 0.8278172016143799),
 ('ноябрь', 0.8244150280952454),
 ('июль', 0.822899580001831),
 ('август', 0.8112362623214722),
 ('январь', 0.80228590965271)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [169]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        if len(pairs) == 1:
            neighbours  = ru_emb.most_similar(mapped_vectors)
        else:
            neighbours  = ru_emb.most_similar([mapped_vectors[i]])

        for j in range(topn):
            key_ru, _ = neighbours[j]
            if key_ru == ru:               
                num_matches += 1 

    precision_val = num_matches / len(pairs)
    return precision_val

In [170]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [171]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [172]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [173]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [ ]:
import numpy as np

In [177]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE GOES HERE
    # compute orthogonal embedding space mapping
    # mapping = ...
    u, s, vh = np.linalg.svd(X_train.T @ Y_train)
    mapping = u @ vh
    return mapping

In [178]:
W = learn_transform(X_train, Y_train)

In [179]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.805662989616394),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938189506530762),
 ('февраль', 0.7923861145973206),
 ('декабрь', 0.7715375423431396)]

In [180]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [181]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2022-12-11 14:24:05--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M   904KB/s    in 2.0s    

2022-12-11 14:24:09 (904 KB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [182]:
!gzip -d ./uk.txt.gz

In [183]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [184]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [186]:
uk_corpus[:10]

['Я вже закінчу коледж, коли ви вернетеся з Америки.\n',
 'Він наказав мені негайно вийти з кімнати.\n',
 'Як би ти не намагався, ти не вивчиш англійську за два-три місяці.\n',
 'Поки я не подзвонив, він не прийшов.\n',
 'У всесвіті багато галактик.\n',
 'Вона приймає душ щоранку.\n',
 'Неслухняний хлопчик заблукав й оглядався по сторонах.\n',
 'Вона повільно зникала в туманному лісі.\n',
 'Наш літак летів понад хмарами.\n',
 'У Майка є декілька друзів у Флориді.\n']

In [ ]:
# Any necessary preprocessing if needed
# YOUR CODE HERE

In [201]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
    translated = []
    rus_word = ''
    list_words = sentence.split()
    for uk_word in list_words:
        try:
            uk_embeding = uk_emb[uk_word]
            ns = ru_emb.most_similar([np.matmul(uk_emb[uk_word], W)])
            rus_word, _ = ns[0]
        except:
            rus_word = 'UNK' 

        translated.append(rus_word)

    return " ".join(translated)

In [202]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [203]:
for sent in uk_corpus[::10]:
    print(translate(sent))

Я уже закончу UNK когда мы прибежишь со океании
Город бомбили враждебные UNK
UNK мной UNK конечно это не UNK что мной не общаюсь со людьми
Впрочем утра выпала UNK
Беда не приходит одна
Посмотри по тот жеребей
Я заказал два UNK
Я не хотел никого UNK
Гора покрыта UNK
по фотографии во девушки корона не со UNK а со цветы
Во меня То UNK
Я приехал во Японию со Китая
по север находится UNK по юге — UNK по востоке — UNK и ещe дальше по востоке — северная UNK
Его родная страна — UNK
Берн — столица UNK
Он ждал по него к десятой часа
Ты можешь взять ту книгу UNK
Такой роман сочинил известный американский писатель
UNK будте UNK комнату возле международного аэропорта во UNK
Он UNK что ты его UNK
Я UNK что ты UNK
UNK кто всё UNK UNK
Во этой реке опасно UNK
UNK UNK UNK
Я хожу к школы UNK
Не моя UNK
Не забудь UNK
Кто UNK
Вы будете чай ли UNK
Он не пойдет по UNK как и мной
Когда Вы UNK
Это моя любимая песня
мы почти UNK
Какой красивый сегодня UNK
Я против каких-либо UNK
поверхность воздушной шары — UNK

Great! 
See second notebook for the Neural Machine Translation assignment.